This notebook aims to implement the algorithm described in [Breiman's Stacked regression paper](https://link.springer.com/content/pdf/10.1007/BF00117832.pdf). The idea behind it is basicaly to rely on [ridge regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) to select coefficients for the predictions on the first layer of the stacked model. Ridge is selected because it can generate coefficients that are equal to zero, thus selecting predictions.

Another interesting idea in the paper is to make it so the ridge regression only produces non-negative coefficients. The explanation from the paper is:

> Only partial answers are available. Suppose that the $v_k(x)$ are strongly correlated and the {$\alpha_k$} are chosen using least squares or ridge regression. Then there is no guarantee that the resulting predictor ~k akvk (x) will stay near the range [mink vk (x), maxk vk (x)] and generalization may be poor.

Where $v_k$ is the k-th model on the first layer, $x$ is the original data and $\alpha_k$ is the k-th coefficient from the regression.

For the experiments, we'll use the same data set as the paper (Boston Housing), available on scikit learn.

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

RANDOM_STATE = 3824500

X, y = load_boston(return_X_y=True)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=50, random_state=RANDOM_STATE)

The algorithm described in the paper for growing trees is too complicated for this example, so we'll simplify it. Instead, we'll just use a bunch of [random forest regressors](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) with different configurations.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
import numpy as np

np.random.seed(RANDOM_STATE)

rf_param_grid = {"n_estimators": [10, 50, 100, 300],
                 "max_features": ["auto", "sqrt", "log2"],
                 "max_depth": [1, 3, 9, None],
                 "min_samples_split": [2, 5, 10],
                 "min_samples_leaf": [1, 2, 4],
                 "n_jobs": [1],
                 "random_state": np.random.randint(1000000, size=3)}

tree_configs = ParameterGrid(rf_param_grid)

regs = [RandomForestRegressor(**params) for params in tree_configs]
print("%d Random forest regressors on the first layer" % len(regs))

1296 Random forest regressors on the first layer


Now we'll build the first layer using wolpert. To make it as close as possible to the paper, we'll use a cross validation approach with 10 folds.

In [3]:
from wolpert.wrappers import CVWrapper
from wolpert.pipeline import make_stack_layer
from sklearn.model_selection import KFold

cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
wrapper = CVWrapper(cv=cv, default_method='predict', n_cv_jobs=-1)

layer0 = make_stack_layer(*regs, blending_wrapper=wrapper)

To make things easier, we'll use [scikit learn's ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) as it already implements the non-negative constraint. To make it behave as a ridge regression, all we have to do is remove the L1 regularization.

In [8]:
from sklearn.linear_model import ElasticNet
from wolpert.pipeline import StackingPipeline

meta_reg = ElasticNet(l1_ratio=0., random_state=RANDOM_STATE, positive=True)

final_reg = StackingPipeline([("l0", layer0), ("meta", meta_reg)])

In [9]:
final_reg.fit(Xtrain, ytrain)

/home/caio/anaconda3/envs/wolpert_devel/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


StackingPipeline(memory=None,
         steps=[('l0', StackingLayer(n_jobs=1,
       transformer_list=[('randomforestregressor-1', CVStackableTransformer(cv=KFold(n_splits=10, random_state=3824500, shuffle=True),
            estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
           max_features='auto', max_l...compute=False,
      random_state=3824500, selection='cyclic', tol=0.0001,
      warm_start=False))])

Now let's check the MSE on the test set.

In [10]:
from sklearn.metrics import mean_squared_error

preds_test = final_reg.predict(Xtest)

print("Test MSE for stacked ensemble: %.5f" % mean_squared_error(ytest, preds_test))

Test MSE for stacked ensemble: 5.36880


We can find the best MSE by checking the blended results from layer 0 against the real values, as each column is the cross validation predictions of a single estimator.

* __Note:__ we have to use the cross validation scores to choose the best estimator on the first layer, not just selecting the one that gives the best one in the test set.

In [17]:
Xt_train, train_indexes = layer0.fit_blend(Xtrain, ytrain)

best = -1
best_mse = 1e20

for i in range(Xt_train.shape[1]):
    candidate_mse = mean_squared_error(ytrain[train_indexes], Xt_train[:, i])
    if candidate_mse < best_mse:
        best_mse = candidate_mse
        best = i
        
print("CV MSE for best regressor on first layer is %.5f" % best_mse)

Xt_test = layer0.transform(Xtest)
mse_test = mean_squared_error(ytest, Xt_test[:, best])
print("Test MSE for best regressor on first layer is %.5f" % mse_test)

CV MSE for best regressor on first layer is 11.05074
Test MSE for best regressor on first layer is 5.61604


Just like in the article, the best regressor on the first layer is still worse than the stacked regression.

We can also check that the ridge regressor is selecting a small subset of models by looking at the sum of its coefficients:

In [18]:
sum_coefs = np.sum(final_reg.named_steps["meta"].coef_)

print("Sum of ridge regressor's coefficients: %.5f" % sum_coefs)

Sum of ridge regressor's coefficients: 1.11493


Which means that only around 1% of the base models were used:

In [23]:
(final_reg.named_steps["meta"].coef_ == 0.).sum()

1274